In [2]:
import json
import numpy as np
import pandas as pd
from nltk.stem.snowball import FrenchStemmer
import unicodedata
import re
import math

In [3]:
df=pd.read_csv("Data/locations/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
_insee_communes, counts = np.unique(communes_, return_counts=True)
insee_communes = np.unique([x.lower() for x in _insee_communes])

In [6]:
_dbPedia = json.load(open("Data/dbPedia_extracted_demonyms3.json"))
dbPedia = { x.lower():[v_.lower() for v_ in v] for x,v in _dbPedia.items()}
_infobox = json.load(open("Data/infobox_extracted_demonyms3.json"))
infobox = { x.lower():[v_.lower() for v_ in v] for x,v in _infobox.items()}
habitants_com =  json.load(open("Data/habitants_com_extracted_demonyms4.json"))
_infobox_wikidata = json.load(open("Data/wikidata_infobox_extracted_demonyms_processed.json"))
infobox_wikidata = { x.lower():[v_.lower() for v_ in v] for x,v in _infobox_wikidata.items()}

In [13]:
merged = {}
for c in insee_communes:
    s1 = set([ x.lower() for x in dbPedia.get(c, [])])
    s2 = set([x.lower() for x in infobox_wikidata.get(c, [])])
    s3 = set([x.lower() for x in habitants_com.get(c, [])])
    s = s1.union(s2).union(s3)
    if len(s)>0:
        merged[c] = list(s)
    

In [16]:
x = 0
for c in insee_communes:
    if c in dbPedia and c not in infobox_wikidata:
        print(c)
        x+=1
x

abitain
achen
agde
agny
aigues-mortes
aimargues
aime-la-plagne
aiti
ajaccio
albon-d'ardèche
allemagne-en-provence
allemant
allevard
altenach
alès
amboise
amfreville-saint-amand
amiens
ampriani
ancenis-saint-géréon
ancy-dornot
andoins
andrezel
angers
angous
annecy
anos
anzin
apt
araujuzon
arbonne
arbonne-la-forêt
arbori
arcangues
argelès-gazost
argences en aubrac
argentat-sur-dordogne
arles
arnéguy
arpajon
arras
arros-de-nay
arvière-en-valromey
asnières-sur-seine
aspach-michelbach
assat
athis-val de rouvre
aubagne
aubertin
aubervilliers
aubignosc
aunac-sur-charente
auneau-bleury-saint-symphorien
aurseulles
aussevielle
autrans-méaudre en vercors
autun
auvers-sur-oise
auxi-le-château
azérat
aÿ-champagne
bagnoles de l'orne normandie
baigts-de-béarn
bailly-romainvilliers
balansun
baliracq-maumusson
baliros
balleroy-sur-drôme
ballon-saint mars
bar-le-duc
barbizon
barcelonnette
barcy
barfleur
bargème
barinque
barzan
bassillac et auberoche
baugé-en-anjou
bazouges cré sur loir
beaufort-en-anjou

788

In [14]:
len(merged)

30408

In [69]:
# with open("Data/merged.json", "w") as fp:
#         json.dump(merged,fp)

In [70]:
def stem_nltk(word):
    stemmer = FrenchStemmer()
    return stemmer.stem(word)

In [81]:
all = json.load(open("Data/merged.json"))
reversed , reversed_stemmed = {}, {}

In [ ]:
for commune, demonyms in all.items():
    for demonym in demonyms:
        if demonym in reversed:
            reversed[demonym].add(commune)
        else:
            reversed[demonym] = set([commune])

        stemmed = stem_nltk(demonym)
        if stemmed in reversed_stemmed:
            reversed_stemmed[stemmed].add(commune)
        else:
            reversed_stemmed[stemmed] = set([commune])
for k,v in reversed.items():
    reversed[k] = list(v)
for k,v in reversed_stemmed.items():
    reversed_stemmed[k] = list(v)

with open("Data/merged_reversed.json", "w") as fp:
         json.dump(reversed,fp)
with open("Data/merged_reversed_stemmed.json", "w") as fp:
         json.dump(reversed_stemmed,fp)

In [72]:
def get_location_demonym_dict(demonym, stemmed_dict):
    stemmed = stem_nltk(demonym)
    if stemmed in stemmed_dict:
        return stemmed_dict[stemmed]
    return -1

In [73]:
get_location_demonym_dict("bellifontaines", reversed_stemmed)

['beaulieu-les-fontaines', 'belfonds', 'bellefontaine', 'fontainebleau']

In [74]:
print(len(reversed), len(reversed_stemmed))

61689 39568


In [79]:
all = json.load(open("Data/departements_demonyms.json"))
reversed_stemmed = {}

for dep, demonyms in all.items():
    for demonym in demonyms:
        stemmed = stem_nltk(demonym)
        if stemmed in reversed_stemmed:
            reversed_stemmed[stemmed].add(dep.lower())
        else:
            reversed_stemmed[stemmed] = set([dep.lower()])

for k,v in reversed_stemmed.items():
    reversed_stemmed[k] = list(v)

with open("Data/deps_stemmed_reversed.json", "w") as fp:
         json.dump(reversed_stemmed,fp)


In [87]:
dbPedia["val de virvée"]

['albins']